# Books Recommender System Using Clustering | Collaborative-Based

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import gdown
import os

# Function to download a file from Google Drive if it doesn't exist
def download_file(file_id, output):
    # Create the /data/ directory if it doesn't exist
    if not os.path.exists('data'):
        os.makedirs('data')

    # Full path to the file in /data/
    output_path = os.path.join('data', output)

    # Check if the file already exists
    if not os.path.exists(output_path):
        url = f"https://drive.google.com/uc?id={file_id}"
        gdown.download(url, output_path, quiet=False)
        print(f"{output} downloaded.")
    else:
        print(f"{output} already exists.")

# File IDs from Google Drive
books_file = '1U4kz_Y4A9fsnXPleHV_wYJ_yB4XxsJXs'
ratings_file = '1hMgOVMci3iaGLRrUKI-PRMBinPpQIXcP'
users_file = '195Mgo4sKzpJ9vfYqiVg7d_NL4vX4Iexj'

# Download files if they do not already exist in /data/
download_file(books_file, "Books.csv")
download_file(ratings_file, "Ratings.csv")
download_file(users_file, "Users.csv")

Books.csv already exists.
Ratings.csv already exists.
Users.csv already exists.


In [3]:
books = pd.read_csv('data/Books.csv', low_memory=False)

In [4]:
books.sample(5)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
185928,044013417X,"Hand in Hand (Candlelight Ecstasy Romance, No ...",Edith Delatush,1985,Dell Publishing,http://images.amazon.com/images/P/044013417X.0...,http://images.amazon.com/images/P/044013417X.0...,http://images.amazon.com/images/P/044013417X.0...
167872,0295966521,A Guide to Book Publishing,"Datus C., Jr. Smith",1989,Univ of Washington Pr,http://images.amazon.com/images/P/0295966521.0...,http://images.amazon.com/images/P/0295966521.0...,http://images.amazon.com/images/P/0295966521.0...
144123,0006391893,Rebeccas Tale,Sally Beauman,2003,Augsburg Fortress Publishers,http://images.amazon.com/images/P/0006391893.0...,http://images.amazon.com/images/P/0006391893.0...,http://images.amazon.com/images/P/0006391893.0...
239148,1565927311,GIMP Pocket Reference (O'Reilly Pocket Referen...,Sven Neumann,2000,O'Reilly,http://images.amazon.com/images/P/1565927311.0...,http://images.amazon.com/images/P/1565927311.0...,http://images.amazon.com/images/P/1565927311.0...
112302,0099402599,Priceless Riches of Planet Phu (Mad Maps S.),Bambi Smyth,1999,Red Fox,http://images.amazon.com/images/P/0099402599.0...,http://images.amazon.com/images/P/0099402599.0...,http://images.amazon.com/images/P/0099402599.0...


In [5]:
print(books['Year-Of-Publication'].unique())

['2002' '2001' '1991' '1999' '2000' '1993' '1996' '1988' '2004' '1998'
 '1994' '2003' '1997' '1983' '1979' '1995' '1982' '1985' '1992' '1986'
 '1978' '1980' '1952' '1987' '1990' '1981' '1989' '1984' '0' '1968' '1961'
 '1958' '1974' '1976' '1971' '1977' '1975' '1965' '1941' '1970' '1962'
 '1973' '1972' '1960' '1966' '1920' '1956' '1959' '1953' '1951' '1942'
 '1963' '1964' '1969' '1954' '1950' '1967' '2005' '1957' '1940' '1937'
 '1955' '1946' '1936' '1930' '2011' '1925' '1948' '1943' '1947' '1945'
 '1923' '2020' '1939' '1926' '1938' '2030' '1911' '1904' '1949' '1932'
 '1928' '1929' '1927' '1931' '1914' '2050' '1934' '1910' '1933' '1902'
 '1924' '1921' '1900' '2038' '2026' '1944' '1917' '1901' '2010' '1908'
 '1906' '1935' '1806' '2021' '2012' '2006' 'DK Publishing Inc' 'Gallimard'
 '1909' '2008' '1378' '1919' '1922' '1897' '2024' '1376' '2037']


In [6]:
books['Year-Of-Publication'].isnull().sum()

0

In [7]:
books[pd.to_numeric(books['Year-Of-Publication'], errors='coerce').isnull()]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,NaN
220731,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-M...",2003,Gallimard,http://images.amazon.com/images/P/2070426769.0...,http://images.amazon.com/images/P/2070426769.0...,http://images.amazon.com/images/P/2070426769.0...,NaN
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,NaN


In [8]:
books['Year-Of-Publication'] = pd.to_numeric(books['Year-Of-Publication'], errors='coerce')

In [9]:
print(books['Year-Of-Publication'].isnull().sum())

3


In [10]:
books['Year-Of-Publication'].fillna(0, inplace=True)

C:\Users\Ziad\AppData\Local\Temp\ipykernel_2240\1793630860.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  books['Year-Of-Publication'].fillna(0, inplace=True)


In [11]:
books[books['Year-Of-Publication'] == 0]['Year-Of-Publication'].count()

4621

In [12]:
print(books['Year-Of-Publication'].isnull().sum())

0


In [13]:
books.sample(5)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
119058,067352440X,American Foreign Policy: Theoretical Essays,G. John Ikenberry,1996.0,Addison-Wesley Pub Co,http://images.amazon.com/images/P/067352440X.0...,http://images.amazon.com/images/P/067352440X.0...,http://images.amazon.com/images/P/067352440X.0...
201709,006107196X,Digimon Digital Monsters: Return to Infinity M...,J. E. Bright,2000.0,HarperEntertainment,http://images.amazon.com/images/P/006107196X.0...,http://images.amazon.com/images/P/006107196X.0...,http://images.amazon.com/images/P/006107196X.0...
13329,9722110500,A varanda do frangipani: Romance (Uma terra se...,Mia Couto,1996.0,Caminho,http://images.amazon.com/images/P/9722110500.0...,http://images.amazon.com/images/P/9722110500.0...,http://images.amazon.com/images/P/9722110500.0...
79872,9507423931,"Cliente, El",John Grisham,1995.0,Planeta,http://images.amazon.com/images/P/9507423931.0...,http://images.amazon.com/images/P/9507423931.0...,http://images.amazon.com/images/P/9507423931.0...
92430,044177427X,Songs of Muad'Dib: Poems and Songs from Frank ...,Frank Herbert,1992.0,Berkley Pub Group,http://images.amazon.com/images/P/044177427X.0...,http://images.amazon.com/images/P/044177427X.0...,http://images.amazon.com/images/P/044177427X.0...


In [14]:
books['Year-Of-Publication'].dtype

dtype('float64')

In [15]:
books['Year-Of-Publication'] = books['Year-Of-Publication'].dropna().astype(int)

In [16]:
books['Year-Of-Publication'].dtype

dtype('int32')

In [ ]:
books.sample(5)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
48404,0803270399,Murders at Moon Dance,"A. B., Jr. Guthrie",1993,University of Nebraska Press,http://images.amazon.com/images/P/0803270399.0...,http://images.amazon.com/images/P/0803270399.0...,http://images.amazon.com/images/P/0803270399.0...
103188,0486233340,Self-Working Card Tricks: 72 Foolproof Card Mi...,Karl Fulves,1976,Dover Publications,http://images.amazon.com/images/P/0486233340.0...,http://images.amazon.com/images/P/0486233340.0...,http://images.amazon.com/images/P/0486233340.0...
1249,0373058446,"Hand Of An Angel (Silhouette Desire, No 844)",BJ James,1994,Silhouette,http://images.amazon.com/images/P/0373058446.0...,http://images.amazon.com/images/P/0373058446.0...,http://images.amazon.com/images/P/0373058446.0...
115564,0473083817,Smarter Than Jack: 87 True Stories,Jenny Campbell,2002,Addenda,http://images.amazon.com/images/P/0473083817.0...,http://images.amazon.com/images/P/0473083817.0...,http://images.amazon.com/images/P/0473083817.0...
204708,0941807304,Funny Kittens (Welcome Books (Steward Tabori &...,J. C. Suares,2000,Welcome Books,http://images.amazon.com/images/P/0941807304.0...,http://images.amazon.com/images/P/0941807304.0...,http://images.amazon.com/images/P/0941807304.0...


In [18]:
books.shape

(271360, 8)

In [19]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [20]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-M']]

In [21]:
books.sample(5)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-M
73157,0962368342,"Mala Noche: And Other \Illegal\"" Adventures""",Walt Curtis,1997,Bridge City Books,http://images.amazon.com/images/P/0962368342.0...
165527,1850746710,Pomeroy Castle,Berry,1995,English Heritage Publications,http://images.amazon.com/images/P/1850746710.0...
56217,0688068898,Echoes in the Darkness,Joseph Wambaugh,1987,Harpercollins,http://images.amazon.com/images/P/0688068898.0...
7464,0375422412,Three Junes,Julia Glass,2002,Pantheon Books,http://images.amazon.com/images/P/0375422412.0...
91045,0816735794,"The Spell of the Black Stone (Welcome Inn, No 4)",E.L. Flood,1995,Troll Communications,http://images.amazon.com/images/P/0816735794.0...


In [22]:
books = books.rename(columns={
    'Book-Title': 'title',
    'Book-Author': 'author',
    'Year-Of-Publication': 'year',
    'Publisher': 'publisher',
    'Image-URL-M': 'image-url'
})

In [23]:
books.sample(5)

,ISBN,title,author,year,publisher,image-url
86325,0452273382,Gossip,Christopher Bram,1998,Plume Books,http://images.amazon.com/images/P/0452273382.0...
263130,0440401364,Lassie Come-Home (Yearling Classics),Eric Knight,1989,Yearling Books,http://images.amazon.com/images/P/0440401364.0...
22486,0671738232,Goodnight Kiss (Fear Street Super Chiller),R. L. Stine,1992,Simon Pulse,http://images.amazon.com/images/P/0671738232.0...
205802,0452284597,Measuring America: How the United States Was S...,Andro Linklater,2003,Plume Books,http://images.amazon.com/images/P/0452284597.0...
201184,3608936238,Du bist zu schnell.,Zoran Drvenkar,2003,Klett-Cotta,http://images.amazon.com/images/P/3608936238.0...


In [24]:
users = pd.read_csv('data/Users.csv')
ratings = pd.read_csv('data/Ratings.csv')

In [25]:
users.sample(5)

,User-ID,Location,Age
190818,190819,"ferndale, washington, usa",NaN
234952,234953,"glendale, california, usa",NaN
149366,149367,"highland park, new jersey, usa",27.0
119754,119755,"frierson, louisiana, usa",40.0
214580,214581,"burnaby, british columbia, canada",NaN


In [26]:
ratings.sample(5)

,User-ID,ISBN,Book-Rating
67506,13935,0515134864,0
147239,32989,1581121717,0
121426,27875,0890244251,9
457645,110029,014050477X,0
62569,12650,7427007752,7


In [27]:
print(f"Shape of books: {books.shape}")
print(f"Shape of users: {users.shape}")
print(f"Shape of ratings: {ratings.shape}")

Shape of books: (271360, 6)
Shape of users: (278858, 3)
Shape of ratings: (1149780, 3)


In [28]:
users = users.rename(columns={
    'User-ID': 'user-id',
    'Location': 'location',
    'Age': 'age'
})
ratings = ratings.rename(columns={
    'User-ID': 'user-id',
    'Book-Rating': 'rating'
})

In [ ]:
len(ratings['user-id'].unique())

105283

In [ ]:
x = ratings['user-id'].value_counts() > 200
x[x]

user-id
11676     True
198711    True
153662    True
98391     True
35859     True
          ... 
274808    True
28634     True
59727     True
268622    True
188951    True
Name: count, Length: 899, dtype: bool

In [31]:
ratings = ratings[ratings['user-id'].isin(x[x].index)]

In [32]:
ratings.shape

(526356, 3)

In [33]:
len(ratings['user-id'].unique())

899

In [ ]:
ratings.sample(5)

,user-id,ISBN,rating
1035762,247447,0671014196,0
464115,110973,0441008968,0
1021122,244736,0425162443,0
1053301,251613,039482167X,0
410901,98391,0843950307,9


In [35]:
ratings_with_books = ratings.merge(books, on='ISBN')

In [36]:
ratings_with_books.shape

(487671, 8)

In [37]:
ratings_with_books.sample(5)

,user-id,ISBN,rating,title,author,year,publisher,image-url
93944,52199,1561381918,9,Gardenspells: An Enchanting Collection of Vict...,Claire Nahmad,1994,Running Pr,http://images.amazon.com/images/P/1561381918.0...
198146,113519,0374121230,0,Child of My Heart,Alice McDermott,2002,Farrar Straus &amp; Giroux,http://images.amazon.com/images/P/0374121230.0...
417338,235842,0440222656,0,The Horse Whisperer,Nicholas Evans,1996,Dell,http://images.amazon.com/images/P/0440222656.0...
464431,261829,0312995423,0,Digital Fortress : A Thriller,Dan Brown,2003,St. Martin's Press,http://images.amazon.com/images/P/0312995423.0...
345507,198711,1557230080,0,Bambi (Disney),Walt Disney,1991,Walt Disney Records &amp; Audio,http://images.amazon.com/images/P/1557230080.0...


In [38]:
num_ratings = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [39]:
num_ratings.sample(5)

,title,rating
138427,The Soldier's Return,1
85655,Northern Stars: The Anthology of Canadian Scie...,2
100399,SPELLBOUND : SPELLBOUND,13
115395,The Age of Consent,3
79384,Modernismen (Bezige Bij pocket),1


In [40]:
num_ratings = num_ratings.rename(columns={
    'rating': 'num-of-ratings'
})

In [41]:
num_ratings.sample(5)

,title,num-of-ratings
52021,Hamlet (New Shakespeare S.),2
75798,Marina and Lee,1
25802,Come a Stranger,7
31908,Death in Springtime,1
31072,Dead Air: A Novel,9


In [42]:
final_ratings = ratings_with_books.merge(num_ratings, on='title')

In [43]:
final_ratings.sample(5)

,user-id,ISBN,rating,title,author,year,publisher,image-url,num-of-ratings
215449,125692,0671461494,0,The Hitchhiker's Guide to the Galaxy,Douglas Adams,1982,Pocket,http://images.amazon.com/images/P/0671461494.0...,84
19535,11676,0425189465,9,Chocolate Quake,Nancy Fairbanks,2003,Berkley Publishing Group,http://images.amazon.com/images/P/0425189465.0...,10
59327,31556,0373832818,0,Eden (Calloway Corners) (Calloway Corners),Penny Richards,1993,Harlequin,http://images.amazon.com/images/P/0373832818.0...,1
204012,115120,031205436X,0,Generation X: Tales for an Accelerated Culture,Douglas Coupland,1992,St. Martin's Press,http://images.amazon.com/images/P/031205436X.0...,17
52463,28360,0553139940,6,Deadwood City (Choose Your Own Adventure #8),Edward Packard,1980,Bantam Doubleday Dell,http://images.amazon.com/images/P/0553139940.0...,4


In [44]:
final_ratings.shape

(487671, 9)

In [45]:
final_ratings = final_ratings[final_ratings['num-of-ratings'] >= 50]

In [46]:
final_ratings.sample(5)

,user-id,ISBN,rating,title,author,year,publisher,image-url,num-of-ratings
82599,40943,0425167313,0,Here on Earth,Alice Hoffman,1998,Berkley Publishing Group,http://images.amazon.com/images/P/0425167313.0...,88
163934,95359,140003065X,0,A Fine Balance,Rohinton Mistry,2001,Vintage Books USA,http://images.amazon.com/images/P/140003065X.0...,63
402270,230708,0440212561,0,Outlander,DIANA GABALDON,1992,Dell,http://images.amazon.com/images/P/0440212561.0...,113
428926,241666,0380717018,8,Body of Evidence (Kay Scarpetta Mysteries (Pap...,Patricia D. Cornwell,1992,Avon,http://images.amazon.com/images/P/0380717018.0...,69
387032,225232,0671867156,6,Pretend You Don't See Her,Mary Higgins Clark,1998,Pocket,http://images.amazon.com/images/P/0671867156.0...,105


In [47]:
final_ratings.shape

(61853, 9)

In [48]:
final_ratings = final_ratings.drop_duplicates(['title', 'user-id'])

In [49]:
final_ratings.shape

(59850, 9)

In [50]:
book_pivot = final_ratings.pivot_table(columns='user-id', index='title', values='rating')

In [51]:
book_pivot

user-id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [52]:
book_pivot = book_pivot.fillna(0)

In [53]:
book_pivot

user-id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
from scipy.sparse import csr_matrix

In [55]:
book_sparse = csr_matrix(book_pivot)

In [56]:
book_sparse

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 14961 stored elements and shape (742, 888)>

In [57]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')

In [58]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [59]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1, -1), n_neighbors=6)

In [60]:
distance

array([[ 0.        , 67.75691847, 68.05145112, 72.277244  , 75.81556568,
        76.30203143]])

In [61]:
suggestion

array([[237, 238, 240, 241, 184, 536]], dtype=int64)

In [62]:
for book in suggestion[0]:
    print(book_pivot.index[book])

Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Sorcerer's Stone (Book 1)
Exclusive
The Cradle Will Fall


In [63]:
book_pivot.index[237]

'Harry Potter and the Chamber of Secrets (Book 2)'

In [64]:
book_name = book_pivot.index

In [65]:
book_name

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [66]:
import pickle
pickle.dump(model, open('artifacts/model.pkl', 'wb'))
pickle.dump(book_name, open('artifacts/book_name.pkl', 'wb'))
pickle.dump(final_ratings, open('artifacts/final_ratings.pkl', 'wb'))
pickle.dump(book_pivot, open('artifacts/book_pivot.pkl', 'wb'))

In [67]:
def recommend_book(book_name, n_neighbors=6):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1, -1), n_neighbors=n_neighbors+1)
    for book in suggestion[0]:
        print(book_pivot.index[book])
    

recommended_books = recommend_book(book_name='A Bend in the Road', n_neighbors=10)

A Bend in the Road
Exclusive
The Cradle Will Fall
No Safe Place
Family Album
Last Man Standing
Lake Wobegon days
Long After Midnight
Abduction
Jacob Have I Loved
Fine Things
